In [2]:
import os
import numpy as np

from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.stem import PorterStemmer

#preprocessing
       
files_name=natsorted(os.listdir('files'))
document_of_terms=[]
for files in files_name:
    with open(f'files/{files}','r') as f:
        document=f.read()
        
         
    tokenized_documents=word_tokenize(document)
    terms=[]
    for word in tokenized_documents:
            terms.append(word)
    document_of_terms.append(terms)

print('                                               *****    Documents in  Terms   *****                                                                        \n    '
    ,  document_of_terms,'\n')                        

#stemming 
stemmer=PorterStemmer()
document_of_stemms=[]
for terms in document_of_terms:
     for word in terms :
         stemmed_terms=[stemmer.stem(word)]
         document_of_stemms.append(stemmed_terms)

print('                                               *****     Stemmed Terms   *****                                                                        \n    '
    ,  document_of_stemms,'\n')        

#positional_index

document_number = 0
positional_index = {}

print() 
print("                                               *****     positions    *****                                       \n   ")
for document in document_of_terms:
    for positional, term in enumerate(document):
        if term in positional_index:
            positional_index[term][0] = positional_index[term][0] + 1

            if document_number in positional_index[term][1]:
                positional_index[term][1][document_number].append(positional)
            else:
                positional_index[term][1][document_number] = [positional]

        else:
            positional_index[term] = []
            positional_index[term].append(1)
            positional_index[term].append({})
            positional_index[term][1][document_number] = [positional]

    document_number += 1
    

print(positional_index,'\n')


#query_preprocessing

query=input('put query here:')

final_list=[[]for i in range (10)]
print('                                    ***** all response list *****                                                     ')
for word in query.split():    
    if word in positional_index.keys():
        for key in positional_index[word][1].keys():
            
            if final_list[key]!=[]:
                
                 if final_list[key][-1] == positional_index[word][1][key][0]-1:
                        final_list[key].append(positional_index[word][1][key][0])
        
            else:
                 final_list[key].append(positional_index[word][1][key][0])
            
print(final_list,'\n')

print('                                  *****response list to query*****                                              ')
for position,list in enumerate(final_list,start=1):
    if len(list)==len(query.split()):
      print(position,list)     



#find tf and wtf
import pandas as pd
import math


all_words=[]
for doc in document_of_terms:
    for word in doc:
        all_words.append(word)
 #tf       
def get_term_freq(doc):
    words_found=dict.fromkeys(all_words,0)
    for word in doc:
        words_found[word]+=1
    return words_found

term_freq=pd.DataFrame(get_term_freq(document_of_terms[0]).values(),index=get_term_freq(document_of_terms[0]).keys())              
print('                                  *****term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)):
    term_freq[i]=get_term_freq(document_of_terms[i]).values()
    
term_freq.columns=['doc'+str(i) for i in range (1,11)]    
print(term_freq)     

#wtf

def get_weighted_term_freq(x):
    if x>0:
      return math.log(x)+1
    return 0
     
print('                                  ***** weighted term freq for each  term in doc*****                                              ')
print()        
for i in range(1,len(document_of_terms)+1):
    term_freq['doc'+str(i)]= term_freq['doc'+str(i)].apply(get_weighted_term_freq) 
    
print(term_freq,'\n')
print()
 
#idf and tf*idf
 
tfd=pd.DataFrame(columns=['df','idf'])

print('                                  *****        IDF          *****                                              ')
for i in range(len(term_freq)):
    
    frequancy=term_freq.iloc[i].values.sum()
    
    tfd.loc[i,'df']=frequancy
    tfd.loc[i,'idf']=math.log10(10/(float(frequancy)))
    
tfd.index=term_freq.index
    
print(tfd)

# tf*idf

tf_idf=term_freq.multiply(tfd['idf'],axis=0)

print('                                  *****         TF*IDF             *****                                              ')
print()      
print(tf_idf)

# doc length

def get_doc_len(col):
    return np.sqrt(tf_idf[col].apply(lambda x:x**2).sum())

doc_len=pd.DataFrame()
for col in tf_idf.columns:
    doc_len.loc['length', col+'__length']=get_doc_len(col)   
print('                                  *****         Doc Length             *****                                              ') 
print(doc_len,'\n')    



# normalization
norm_tf_idf= tf_idf.divide(doc_len.values[0],axis=1)
print('                                  *****        Normalized TF*IDF           *****                                              ')
print()
print(norm_tf_idf)

                                               *****    Documents in  Terms   *****                                                                        
     [['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'], ['antony', 'brutus', 'caeser', 'calpurnia'], ['mercy', 'worser'], ['brutus', 'caeser', 'mercy', 'worser'], ['caeser', 'mercy', 'worser'], ['antony', 'caeser', 'mercy'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where'], ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']] 

                                               *****     Stemmed Terms   *****                                                                        
     [['antoni'], ['brutu'], ['caeser'], ['cleopatra'], ['merci'], ['worser'], ['antoni'], ['brutu'], ['caeser'], ['calpurnia'], ['merci'], ['worser'], ['brutu'], ['caeser'], ['merci'], ['worser'], ['ca

#stack sol for bool query

In [3]:
python_query="foo OR bar AND NOT gazonk"
sql_keywords=["AND","NOT","OR"]
sql_query=[]
for word in python_query.split(" "):
    if word in sql_keywords:
        sql_query+=[ word ]
    else:
        sql_query+=["dictkey='%s'" % word]

real_sql_query=" ".join(sql_query)

In [19]:
def search(self, sq: BoolQuery) -> list:

    # Performs a search from a given query in boolean retrieval model,
    # Supports AND queries only and returns sorted document ID's as result:

    if sq.is_empty():
        return super().search(sq)

    terms = [self.index[term] for term in sq.get_terms() if term in self.index]
    if not terms:
        return []

    # Iterate over posting lists and intersect:
    result, terms = terms[0].pst_list, terms[1:]
    while terms and result:
        result = self.intersect(result, terms[0].pst_list)
        terms = terms[1:]
    return [p.id for p in result]

NameError: name 'BoolQuery' is not defined

In [20]:
def intersect(p1: list, p2: list) -> list:

    # Performs linear merge of 2x sorted lists of postings,
    # Returns the intersection between them (== matched documents):

    res, i, j = list(), 0, 0
    while i < len(p1) and j < len(p2):
        if p1[i].id == p2[j].id:
            res.append(p1[i])
            i, j = i + 1, j + 1
        elif p1[i].id < p2[j].id:
            i += 1
        else:
            j += 1
    return res

In [4]:
def intersection(list1, list2):
    return list(set(list1).intersection(list2))
def union(list1, list2):
    return list(set(list1).union(list2))
def notin(list1, list2):
    return [filter(lambda x: x not in list1, sublist) for sublist in list2]

#intersection(positional_index['fools'], intersection(positional_index['fear'], positional_index['angels']))

# try to use function

In [ ]:
list1=[1,3,4,7,8,10]
list2=[5,7,4,8,10]
list3=[1,2,3,4,5]

In [9]:
x=intersection(list1,list2)+notin(list2,list3)
print (x)

TypeError: 'int' object is not iterable

# inverted index

In [10]:
import re

query=input('write your query:')

#query="fools OR fear  AND NOT angle"

result_set=set()
operation=None

for word in re.split(" +(AND|OR) +",query):
    #word will be in ['foo', 'OR', 'bar', 'AND', 'NOT gazonk']

    inverted=False # for "NOT word" operations

    if word in ['AND','OR']:
        operation=word
        continue

    if word.find('NOT ') == 0:
        if operation== 'OR':
        # generally "OR NOT" operation does not make sense, but if it does in your case, you 
        # should update this if() accordingly
            continue

        inverted=True
        # the word is inverted!
        realword=word[4:]
    else:
        realword=word

    if operation is not None:
         # now we need to match the key and the filenames it contains:
        current_set=set(inverted_index[realword])

        if operation == 'AND':
            if inverted == True:
                result_set -= current_set
            else:
                result_set &= current_set
        elif operation == 'OR':
            result_set |= current_set

    operation=None

print (result_set)

<>:20: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:20: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?


set()


C:\Users\future\AppData\Local\Temp\ipykernel_14332\2260377334.py:20: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if operation is 'OR':
